In [1]:
from itertools import combinations 
from random import shuffle 
import numpy as np 
import sys

In [2]:
# general stuff that will work for any card game 

class Card:
    
    def __repr__( self ):
        return '{}{}'.format( self.display_rank, self.display_suit )
    
    def __init__( self, suit, rank ):
        display_suits = [ 's', 'h', 'c', 'd' ]
        display_ranks = [ 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K' ]
        vals = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10 ]
        map_suits = dict(zip(range(4),display_suits))
        map_ranks = dict(zip(range(13),display_ranks))
        map_vals = dict(zip(range(13),vals))

        self.suit = suit
        self.rank = rank 
        self.index = (13*self.suit)+self.rank
        self.display_rank = map_ranks[ self.rank ]
        self.display_suit = map_suits[ self.suit ]
        self.value = map_vals[ self.rank ]
        
class Deck:
    def __init__( self ):
        self.cards = []
        for s in range( 4 ):
            for r in range( 13 ):
                self.cards.append(Card(s, r))
                
    def shuffle( self ):
        shuffle( self.cards )
        
    def draw( self, n ):
        result = []
        for i in range( n or 1 ):
            result.append( self.cards.pop() ) 
        return result 

In [3]:
deck = Deck()
deck.shuffle()
deck.draw(5)

[3c, 2d, 9d, 9c, 6s]

In [4]:
# def play_a_game():
#     '''
#     Plays a game of cribbage. Returns a winner, 0 or 1
#     '''
#     sc = np.array([0,0]) 
#     hand_count = 0 
#     while True:
#         hand_count += 1
#         print( '>>> Hand count:', hand_count ) 
#         print( '>>> Game score:', sc )
#         outcome = play_a_hand()
#         sc += np.array(outcome)
#         for i, p in enumerate( sc ):
#             if p > 120:
#                 return i 
        
# play_a_game()

In [5]:
# # play 10000 games 

# features = []
# target = []
# n_games = 10
# for n in range( n_games ): 
#     outcome, play_vector = play_a_game()
#     target.append( outcome ) 
#     features.append( play_vector ) 
#     sys.stdout.write("\rPlaying game {} of {} games".format( n+1, n_games ) )
#     sys.stdout.flush()
# sys.stdout.write("\rPlayed {0}/{0} games. Done.  ".format( n_games ) )
# sys.stdout.flush()

In [6]:
# %matplotlib inline 
# import matplotlib.pyplot as plt
# from sklearn import preprocessing, metrics, svm, linear_model, model_selection

In [7]:
# features = np.array(features)
# # features.shape

In [8]:
# one_hot = preprocessing.OneHotEncoder()
# transformed = one_hot.fit_transform( features ) 
# transformed.shape

In [9]:
# for i in range( 10 ):
#     X_train, X_test, y_train, y_test = model_selection.train_test_split(transformed, target, test_size=0.33)
#     clf = svm.SVC(kernel='linear',probability=True)
#     clf.fit( X_train, y_train ) 
#     pred = clf.predict( X_test ) 
#     prob = clf.predict_proba( X_test )
#     fpr, tpr, __ = metrics.roc_curve( y_test, prob[:,0] ) 
#     plt.plot( fpr, tpr ) 

1. First, see if there's a mvove th computer can make that will win the game (i.e. make the score 31). If there is, take that move. Otherwise, go to setp 2. 

2. See if there's a move the player could make that would cause the computer to lose the game (i.e. play a card such that computer cannot play). If there is, block that move. 

3. Play a predefined good/safe card. 

4. Play a different predefined safe card. 

5. Play the only legal card. 

6. Say "go" . 

In [37]:
# players 

class Player():
    def __init__( self, name ): 
        self.name = name 
        self.hand = []
        self.score = 0
    def ask_for_discards( self ):
        return [self.hand.pop(), self.hand.pop()] 
    def clean( self ):
        self.hand = []
        self.score = 0 
            
class HumanPlayer(Player):
    def ask_for_input( self, hand ):
        d = dict(enumerate(hand,1))
        inp = input( 'Choose one of {}'.format( d.values() ) )
        card = d[ int(inp) ]
        return card 

class NondeterministicAIPlayer(Player):
    def ask_for_input(self, hand):
        card = np.random.choice(hand) 
        return card
    
# class NondeterministicAIPlayer(Player):
class AIPlayer(Player):
    def ask_for_input(self, hand):
        card = np.random.choice(hand) 
        # not a very smart AI
        # can it fool will? 
        return card

In [38]:
def discards( players ):
    crib = []
    for player in players:            
        discards = player.ask_for_discards()
        for c in discards:
            crib.append(c) 
    return crib 

In [39]:
def counting( players ):
    
    # counting functions 
    def play( card ):
        nonlocal count, turn, hands
        plays.append( card ) 
        count += card.value
        hands[turn] = [ n for n in hands[turn] if n != card ]         
        #print( card, 'was played by player', turn ) 
        return count, hands 

    def say_go():
        nonlocal score 
        pass 
    
    def validate_play( card, count ):
        if card.value + count < 32:
            return True 
        else:
            return False 
    
    # game logic         
    hands = [player.hand for player in players]
    turn = np.random.randint(2) # either 0 or 1 
    count = 0
    plays = []
    play_runs = []
    score = np.array([0,0]) # get a player's score like score[player_number] 

    while all(hands): # while all hands are not empty 
        game_on = True
        while game_on: # we are counting toward 31 
            #print( count, tuple(score) ) 

            # first, does whomever's turn it is even have cards? 
            if not hands[turn]:
                break # out of while game_on 
            # first, is whoever's turn it is even have a legal move
            # or do they have to say "go"? 
            if all([count+card.value>31 for card in hands[turn]]):
                # they have to say "go" 
                # other person gets 1 point 
                score[turn^1] += 1 
                #print( "Go --- player", turn ) 
                break # out of "while game_on" 

            # generic turn 
            done = False
            while not done: 
                card = players[turn].ask_for_input(hands[turn])
                done = validate_play(card, count)
            play(card) 

            # score and set up for next iteration 
            # current player is 
            if count == 31:
                # we are done, son 
                score[ turn ] += 2 # 31 for 2 
                break 
            if count == 15:
                score[ turn ] += 2 # 15 for 2 

            # swap turn     
            turn ^= 1 # swap turn 

        # anything you want to happen after we stop couting to 31 
        #print( "Plays to 31:", plays ) 
        play_runs.append( plays ) 
        count = 0 
        plays = []
        
    # anything you want to happen after we don't have any cards left 
    # return from "counting" function 
    return score 

In [40]:
# cribbage scoring 
# ai may never know this 
def score( hand ):

    points = 0
    assert len(hand) == 5

    # fifteens  
    for vector_length in [ 2, 3, 4, 5 ]:
        for vector in combinations( hand, vector_length ):
            if sum( [ card.value for card in vector ] ) == 15:
                points += 2 

    # pairs (not necessary to account for more than pairs)
    for i, j in combinations( hand, 2 ):
        if i.rank == j.rank:
            points += 2 

    # runs
    for vector_len in [ 5, 4, 3 ]:
        for vec in combinations( hand, vector_len ):
            vals = [ card.value for card in vec ]
            run = [ n + min( vals ) for n in range( vector_len ) ]
            if sorted( vals ) == run:
                points += vector_len
                break
            break 
                      
    return points 

def scoring(players, turn_card):
    for player in players:
        pool = player.hand + turn_card 
        player.score += score(pool) 

In [46]:
#players = [HumanPlayer('alex'), NondeterministicAIPlayer('will')]
ply = [NondeterministicAIPlayer('p1'), NondeterministicAIPlayer('p2')]

def the_real_game(players):
    
    while players[0].score < 121 and players[1].score < 121:
        turn = np.random.randint(2)
        deck = Deck()
        deck.shuffle()
        for player in players:
            player.hand = deck.draw(6)    
        crib = discards(players)
        counting(players)    
        turn_card = deck.draw(1)
        scoring(players, turn_card)
        players[turn^1].score += score(crib+turn_card)
    
    return_val = [ player.score for player in players ] 
    for player in players:
        player.clean()
    return return_val 
        
[the_real_game(ply) for n in range(50)]

[[119, 121],
 [104, 121],
 [106, 123],
 [116, 127],
 [90, 123],
 [122, 87],
 [120, 125],
 [127, 88],
 [106, 126],
 [122, 94],
 [122, 120],
 [124, 103],
 [128, 123],
 [128, 123],
 [120, 126],
 [128, 98],
 [123, 84],
 [124, 112],
 [116, 129],
 [125, 108],
 [96, 123],
 [60, 123],
 [126, 96],
 [121, 116],
 [94, 132],
 [122, 112],
 [96, 126],
 [99, 127],
 [122, 88],
 [127, 100],
 [122, 111],
 [111, 130],
 [106, 125],
 [121, 118],
 [132, 95],
 [113, 121],
 [99, 125],
 [127, 84],
 [109, 121],
 [128, 115],
 [64, 130],
 [109, 124],
 [98, 132],
 [127, 97],
 [130, 99],
 [128, 102],
 [113, 122],
 [123, 126],
 [126, 116],
 [123, 114]]